# Create Combined Heatmap for significant Complexes

This notebook looks at the significant genes in at least one cancer. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')
prot_enr.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Spliceosome Homo sapiens hsa03040,84/134,3.960162e-41,1.160328e-38,0,0,4.801729,446.703392,RBM25;EIF4A3;DDX42;HNRNPU;PRPF19;USP39;EFTUD2;...,KEGG_2016
1,RNA transport Homo sapiens hsa03013,76/172,6.887335e-24,1.008995e-21,0,0,3.384607,180.509081,CYFIP2;NUP107;NUP188;RPP30;EIF4A3;PHAX;PNN;XPO...,KEGG_2016
2,DNA replication Homo sapiens hsa03030,22/36,1.947423e-11,1.901983e-09,0,0,4.681050,115.443730,RFC5;FEN1;RFC3;PCNA;RFC4;MCM7;RFC1;RFC2;PRIM1;...,KEGG_2016
3,Mismatch repair Homo sapiens hsa03430,16/23,6.740173e-10,4.937177e-08,0,0,5.328626,112.528681,RFC5;RFC3;PCNA;RFC4;RFC1;RFC2;RPA1;MLH1;POLD3;...,KEGG_2016
4,mRNA surveillance pathway Homo sapiens hsa03015,34/91,3.983668e-09,2.334429e-07,0,0,2.861941,55.352977,SMG1;RNMT;EIF4A3;MSI1;PPP2R2A;MSI2;SMG7;SMG6;P...,KEGG_2016


# Step 2: Get the list of significant genes from certain pathways

In [3]:
dna_rep = prot_enr.res2d.Genes[2]
dna_genes = dna_rep.split(';')
mis_repair = prot_enr.res2d.Genes[4]
repair_genes = mis_repair.split(';')
genes = dna_genes +repair_genes 
print('total genes:',len(genes))

total genes: 56


# Step 3: Create HeatMap

Slice out the significant genes in the pathway from sig_df (sig_pval_heatmap.csv).

In [4]:
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

56

In [5]:
# Only include p-values < a certain pval
a = 0.05
plot_df = plot_df.loc[plot_df['P_Value'] <= a]

In [6]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                     plot_width=1000)

Loading BokehJS ...

# Step 4: Create a HeatMap with both pos and neg differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [7]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [8]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['MCM6', 'MCM4', 'RFC5', 'MCM5', 'MCM2', 'MCM7', 'MCM3', 'RFC2',
       'RFC4', 'SSBP1', 'POLA2', 'PPP2R2A', 'MSI2'], dtype=object)

In [9]:
genes_pn = genes_pn.loc[genes_pn['Proteomics'] != 'SSBP1'] # NOT sig change in med for endo (blue)
genes_pn.Proteomics.unique()

array(['MCM6', 'MCM4', 'RFC5', 'MCM5', 'MCM2', 'MCM7', 'MCM3', 'RFC2',
       'RFC4', 'POLA2', 'PPP2R2A', 'MSI2'], dtype=object)

In [10]:
# Only include p-values < a
genes_pn = genes_pn.loc[genes_pn['P_Value'] <= a]

In [11]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                     plot_height= 600, plot_width=800)

Loading BokehJS ...

# Revised Heatmap 

In [12]:
df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(df.Proteomics.unique()) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')


In [13]:
prot_enr.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Ribosome biogenesis in eukaryotes Homo sapiens...,13/89,1.180367e-09,3.458475e-07,0,0,9.393403,193.104322,UTP6;IMP3;WDR3;HEATR1;NAT10;WDR75;IMP4;PWP2;UT...,KEGG_2016
1,DNA replication Homo sapiens hsa03030,9/36,3.083251e-09,4.516962e-07,0,0,16.077170,315.068832,RFC3;PCNA;RFC4;MCM7;RFC2;MCM3;MCM4;MCM5;MCM2,KEGG_2016
2,B cell receptor signaling pathway Homo sapiens...,11/73,1.626924e-08,1.588963e-06,0,0,9.690349,173.786622,PPP3CB;SYK;CHUK;PIK3CA;PRKCB;INPP5D;BLNK;RAC2;...,KEGG_2016
3,Fc gamma R-mediated phagocytosis Homo sapiens ...,12/93,2.188722e-08,1.603239e-06,0,0,8.297894,146.352976,PAK1;PTPRC;SYK;PIK3CA;MYO10;PRKCB;INPP5D;RAC2;...,KEGG_2016
4,RNA transport Homo sapiens hsa03013,15/172,8.324097e-08,4.877921e-06,0,0,5.608315,91.424098,RANBP2;NUP210;NUP155;NCBP1;NUP133;THOC3;THOC2;...,KEGG_2016


In [14]:
print(prot_enr.res2d.Term[1])
genes = prot_enr.res2d.Genes[1]
genes = genes.split(';')
found = ['TOPBP1', 'TOP2A', 'GINS2', 'GINS4', 'POLA2', 'CHEK1']
for gene in found:
    genes.append(gene)
genes

DNA replication Homo sapiens hsa03030


['RFC3',
 'PCNA',
 'RFC4',
 'MCM7',
 'RFC2',
 'MCM3',
 'MCM4',
 'MCM5',
 'MCM2',
 'TOPBP1',
 'TOP2A',
 'GINS2',
 'GINS4',
 'POLA2',
 'CHEK1']

In [15]:
get = df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_k = df[get] # Keep only genes with pos and neg
genes_k.Proteomics.unique()

array(['MCM4', 'PCNA', 'MCM5', 'MCM2', 'MCM7', 'MCM3', 'RFC3', 'RFC2',
       'RFC4', 'TOPBP1', 'GINS4', 'GINS2', 'TOP2A'], dtype=object)

In [16]:
# Only include p-values < a certain pval
genes_k = genes_k.loc[genes_k['P_Value'] <= a]

In [17]:
p.plotCircleHeatMap(genes_k, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                     plot_height= 400, plot_width=800)

Loading BokehJS ...